<a href="https://colab.research.google.com/github/barassah/ml_experiments/blob/main/Traditional_Models_AppleDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Apple-Twitter-Sentiment-DFE.csv to Apple-Twitter-Sentiment-DFE.csv


In [ ]:
#reading and describing the dataset
dataset=pd.read_csv("Apple-Twitter-Sentiment-DFE.csv", encoding='ISO-8859-1')
dataset.head()

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,date,id,query,sentiment_gold,text
0,623495513,True,golden,10,NaN,3,0.6264,Mon Dec 01 19:30:03 +0000 2014,5.400000e+17,#AAPL OR @Apple,3\nnot_relevant,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,623495514,True,golden,12,NaN,3,0.8129,Mon Dec 01 19:43:51 +0000 2014,5.400000e+17,#AAPL OR @Apple,3\n1,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,623495515,True,golden,10,NaN,3,1.0000,Mon Dec 01 19:50:28 +0000 2014,5.400000e+17,#AAPL OR @Apple,3,My cat only chews @apple cords. Such an #Apple...
3,623495516,True,golden,17,NaN,3,0.5848,Mon Dec 01 20:26:34 +0000 2014,5.400000e+17,#AAPL OR @Apple,3\n1,I agree with @jimcramer that the #IndividualIn...
4,623495517,False,finalized,3,12/12/14 12:14,3,0.6474,Mon Dec 01 20:29:33 +0000 2014,5.400000e+17,#AAPL OR @Apple,NaN,Nobody expects the Spanish Inquisition #AAPL


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _unit_id              3886 non-null   int64  
 1   _golden               3886 non-null   bool   
 2   _unit_state           3886 non-null   object 
 3   _trusted_judgments    3886 non-null   int64  
 4   _last_judgment_at     3783 non-null   object 
 5   sentiment             3886 non-null   object 
 6   sentiment:confidence  3886 non-null   float64
 7   date                  3886 non-null   object 
 8   id                    3886 non-null   float64
 9   query                 3886 non-null   object 
 10  sentiment_gold        103 non-null    object 
 11  text                  3886 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 337.9+ KB


In [ ]:
dataset = dataset[['text']]  # Keep only text column
print(dataset)

                                                   text
0     #AAPL:The 10 best Steve Jobs emails ever...htt...
1     RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2     My cat only chews @apple cords. Such an #Apple...
3     I agree with @jimcramer that the #IndividualIn...
4          Nobody expects the Spanish Inquisition #AAPL
...                                                 ...
3881  (Via FC) Apple Is Warming Up To Social Media -...
3882  RT @MMLXIV: there is no avocado emoji may I as...
3883  @marcbulandr I could not agree more. Between @...
3884  My iPhone 5's photos are no longer downloading...
3885  RT @SwiftKey: We're so excited to be named to ...

[3886 rows x 1 columns]


In [ ]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    # Modified regex to preserve words like "can't" and "don't"
    text = re.sub(r'[^\w\s]+', ' ', text) # remove punctuation but keep letters, numbers, and spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply text cleaning
dataset['text'] = dataset['text'].apply(clean_text)


In [ ]:
# Convert text into numerical features
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(dataset['text'])

In [ ]:
# Cluster into 2 groups (fake vs genuine)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
pseudo_labels = kmeans.fit_predict(X)

# Add pseudo-labels to the dataset
dataset['label'] = pseudo_labels


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, pseudo_labels, test_size=0.3, random_state=42)


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))


SVM Accuracy: 1.0


In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_preds = nb.predict(X_test)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, nb_preds))


Naïve Bayes Accuracy: 1.0


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))


Random Forest Accuracy: 1.0


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))


Logistic Regression Accuracy: 1.0


In [ ]:
# Print classification reports
print("SVM:\n", classification_report(y_test, svm_preds))
print("Naïve Bayes:\n", classification_report(y_test, nb_preds))
print("Random Forest:\n", classification_report(y_test, rf_preds))
print("Logistic Regression:\n", classification_report(y_test, lr_preds))


SVM:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00      1072

    accuracy                           1.00      1166
   macro avg       1.00      1.00      1.00      1166
weighted avg       1.00      1.00      1.00      1166

Naïve Bayes:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00      1072

    accuracy                           1.00      1166
   macro avg       1.00      1.00      1.00      1166
weighted avg       1.00      1.00      1.00      1166

Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00      1072

    accuracy                           1.00      1166
   macro avg       1.00      1.00      1.00      1166
weighted avg       1.00      1.00     